# Overview
More or less a clone of `bold_register_scratch.ipynb` only now we are going to rely on manually alignment instead of the always failing ML models from ANTs. I don't mean to entirely imply that it is ANTs' fault, I might be misusing the models. However, it's proving difficult, and iterating over something that takes so long to align is taking forever. So lets go with the manual alignment.

**steps**
We will load all the data, run motion correction, and then manually align

# Setup and Load Data

In [18]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact, Button, Output
import pickle
import napari
import matplotlib.transforms as mtransforms

t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [11]:
proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
template_folder =  os.path.join(proj_root, "templates/")
output_folder = os.path.join(proj_root, "output/")

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [12]:
bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)

# Functions

In [13]:
def transform_BOLD(bold_frame: np.ndarray, translation: tuple, scale: tuple, rotation: tuple,):
    tx, ty, tz = translation
    sx, sy, sz = scale
    rx, ry, rz = rotation
    translation = np.array([
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0, 1]
    ])

# Translation matrices copypasta from GPT
    scale = np.array([
            [sx, 0,  0,  0],
            [0,  sy, 0,  0],
            [0,  0,  sz, 0],
            [0,  0,  0,  1]
        ])

# I wonder where the origin is here...
    rotation_x = np.array([
        [1, 0, 0, 0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx), np.cos(rx), 0],
        [0, 0, 0, 1]
    ])
    rotation_y = np.array([
        [np.cos(ry), 0, np.sin(ry), 0],
        [0, 1, 0, 0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [0, 0, 0, 1]
    ])
    rotation_z = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz), np.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    rotation_matrix = rotation_x @ rotation_y @ rotation_z
    transformation_matrix = translation @ rotation @ scale

In [14]:
def manual_bold_alignment(bold_seq_vol: np.ndarray, anat_vol: np.ndarray,
                                  dim='x', bold_cmap = 'viridis', anat_cmap = 'gray'):
    def x_coord(slice_idx, frame_idx, opacity, rotation):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('x axis view')

        bold_slice = bold_seq_vol[slice_idx,:,:, frame_idx]

        bold_im = axes[0].imshow(bold_slice, cmap=bold_cmap)
        bold_im.set_transform(mtransforms.Affine2D().rotate_deg(rotation))
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[slice_idx,:,:], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[slice_idx,:,:], cmap=anat_cmap)
        axes[2].imshow(bold_slice, cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

        return rotation
    
    
    def y_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('y axis view')

        axes[0].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap=bold_cmap)
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,slice_idx,:], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,slice_idx,:], cmap=anat_cmap)
        axes[2].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

    def z_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('z axis view')

        axes[0].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap=bold_cmap)
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,:,slice_idx], cmap=anat_cmap)
        axes[2].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

    match dim:
        case "x":
            interact(x_coord, slice_idx=(0, anat_vol.shape[0]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0), rotation=(0,365))
        case 'y':
            interact(y_coord, slice_idx=(0, anat_vol.shape[1]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'z':
            interact(z_coord, slice_idx=(0, anat_vol.shape[2]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))


In [33]:
def generate_brain_mask(anat, mni_template, mni_mask):
    #Don't love the term "anat" as this should work with BOLD, but what is it if not anatomy?
    template_warp_to_raw_anat = ants.registration(
    fixed=anat,
    moving=mni_template, 
    type_of_transform='SyN',
    verbose=False
    )
    print("Creating brain mask")
    brain_mask = ants.apply_transforms(
        fixed=template_warp_to_raw_anat['warpedmovout'],
        moving=mni_mask,
        transformlist=template_warp_to_raw_anat['fwdtransforms'],
        interpolator='nearestNeighbor',
        verbose=False
    )
    return brain_mask

In [34]:
def skull_strip_anat(anat, mni_template, mni_mask, dilate=True):
    """
    anat, mni_template, mni_mask must all be ANTS images.
    """
    print("Skull Stripping Anatomy Volume")
    print("Registering template to frame")
    brain_mask = generate_brain_mask(anat, mni_template, mni_mask)
    if dilate:
        print("Dilating brain mask")
        brain_mask = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')
    print("Masking brain")
    isolated_brain = ants.mask_image(anat, brain_mask)
    print("Done")
    return isolated_brain

In [40]:
def skull_strip_bold(bold, mni_template, mni_mask, dilate=False):
    """
    Assuming a motion corrected or relatively BOLD image
    """
    
    print("Skull strip for BOLD")
    isolated_brain_vol_frames = []
    for frame in range(bold.shape[3]):
        print(f"Skull stripping bold frame {frame + 1}/{bold.shape[3]}")
        bold_frame = ants.from_numpy(bold.numpy()[:,:,:,frame],
                                    spacing=bold.spacing[:3])
        brain_mask = generate_brain_mask(bold_frame, mni_template, mni_mask)
        if dilate:
            print("Dilating brain mask")
            brain_mask = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')
        isolated_brain_vol = ants.mask_image(bold_frame, brain_mask).numpy()
        isolated_brain_vol_frames.append(isolated_brain_vol)
    print("Creating new ANTsImage from isolated brain volumes")
    data = np.stack([frame for frame in isolated_brain_vol_frames], axis=3)
    isolated_brain_bold_img = ants.from_numpy(data, origin=bold.origin, spacing=bold.spacing)
    print("Done")
    return isolated_brain_bold_img
    
        

In [ ]:
def align_stabilized_bold_to_anat(bold_img, t1_img, template_frame_idx=0):
    #This bad boy got a little lost in the copypasta sauce and contains some outdated stuff
    #We really don't need to be returning the whole image, just the numpy array.


    """"
    This function aligns a 4D BOLD image to a T1 anatomy image
    by aligning the mean of the BOLD to the T1 anatomy.
    It uses frame registration from only one "template frame"
    as we are assuming you're using a motion corrected
    BOLD image (or something relatively stable) as your input
    
    It gets things in the ballpark, but does will require some
    extra manual registration afterwards.
    """

    print("Aligning stabilized bold to anat\nEstablishing temporal mean")
    template_frame_idx = ants.from_numpy(bold_img.numpy()[:,:,:,template_frame_idx], spacing=bold_img.spacing[:3])

    frame_registration = ants.registration(
        fixed=t1_img,
        moving=template_frame_idx,
        type_of_transform="Rigid", 
    )
    registered_frames = []

    print("Applying transformations to frames")
    for frame in range(bold_img.shape[3]):
        print(f"frame{frame}/{bold_img.shape[3]}")
        print("creating image of bold frame")
        bold_frame = ants.from_numpy(bold_img.numpy()[:,:,:,frame],
                                     spacing=bold_img.spacing[:3])
        print("     Applying frame transformation")
        registered_frame = ants.apply_transforms(
            fixed=t1_img,
            moving=bold_frame,
            transformlist=frame_registration['fwdtransforms'],
            interpolator='linear'
        )
        print("     adding registered frame to list")
        registered_frames.append(registered_frame)
    print("Creating 4D numpy vol from list of 3D ANTs imgs")
    data = np.stack([frame.numpy() for frame in registered_frames], axis=3)
    print("Creating 4D bold image from numpy vol")
    registered_bold_img = ants.from_numpy(data, origin=bold_img.origin, spacing=bold_img.spacing)
    return registered_bold_img

# Skull Stripping

In [ ]:
isolated_t1 = skull_strip_anat(t1_image, mni_template, mni_mask)

Skull Stripping Anatomy Volume
Registering template to frame
antsRegistration -d 3 -r [0x3242049a8,0x324206148,1] -m mattes[0x3242049a8,0x324206148,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x3242049a8,0x324206148,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmp5jrcxghv,0x324204848,0x324205868] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x3242049a8 and moving image: 0x324206148 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x3242049a8
  moving image: 0x324

In [42]:
# Truncated Version
sliced = isolated_bold.numpy()[:, :, :, :10]
bold_truncated_img = ants.from_numpy(sliced, spacing=bold_image.spacing, origin=bold_image.origin, direction=bold_image.direction)
#stabilized = ants.motion_correction(bold_image)
stabilized_truncated = ants.motion_correction(bold_truncated_img)

In [46]:
print(stabilized_truncated)

{'motion_corrected': ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (64, 64, 35, 10)
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
, 'motion_parameters': [['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmp8dx1pteb0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpvtyu5h7k0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmp6933axa_0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpvy6hcxko0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpauf_q42l0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpm_mmusk10GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpgvd07mk20GenericAffine.mat'], ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmp86_lpund0GenericAffine.mat'], ['/var/folders/m4/rtcmkx_1

In [47]:
isolated_bold = skull_strip_bold(stabilized_truncated['motion_corrected'], mni_template, mni_mask)

Skull strip for BOLD
Skull stripping bold frame 1/10
Creating brain mask
Skull stripping bold frame 2/10
Creating brain mask
Skull stripping bold frame 3/10
Creating brain mask
Skull stripping bold frame 4/10
Creating brain mask
Skull stripping bold frame 5/10
Creating brain mask
Skull stripping bold frame 6/10
Creating brain mask
Skull stripping bold frame 7/10
Creating brain mask
Skull stripping bold frame 8/10
Creating brain mask
Skull stripping bold frame 9/10
Creating brain mask
Skull stripping bold frame 10/10
Creating brain mask
Creating new ANTsImage from isolated brain volumes
Done


Quick check in napari

In [ ]:
viewer = napari.Viewer()
viewer.add_image(np.transpose(isolated_bold.numpy(), axes=(3,0,1,2)), name="BOLD isolated")
viewer.add_image(np.transpose(bold_image.numpy(), axes=(3,0,1,2)), name="BOLD")

<Image layer 'BOLD' at 0x3a5bb3810>

# The Meat of it All

Alignment hasn't been working without skull stripping, so let's reintroduce that


Of note, `AntsPyNet` has a built in function that might be a bit more robust. However, it is heavy and falls into dependency hell quite quickly. It would be worth playing around with, but the heaviness of the package might be antithetical to Neurovolume

In [ ]:
registered_BOLD = align_stabilized_bold_to_anat_p1(stabilized_truncated['motion_corrected'], t1_image)